In [ ]:
import mcstasscript as ms
import make_powder_instrument
import quizlib

In [ ]:
quiz = quizlib.Powder_Quiz()

# Powder diffraction exercise

In this notebook you will work with a McStas model of a simplified powder diffraction instrument.
You will have to answer questions in the notebook by working with this model, both by running simulations and expanding the model.
We will use the Python McStas API McStasScript to work with the instrument, you can find documentation [here](https://mads-bertelsen.github.io).

## Get the instrument object
First we need the McStas instrument object. Here it is retrieved from a local python function that generates it.

In [ ]:
instrument = make_powder_instrument.make()

## Investigate instrument
The first task is to investigate the instrument object `instrument` using some of the available methods available on that object. Each method that show something about the instrument starts with the word show, so you can use tab to autocomplete in the cell to see the relevant methods.

In particular, look at what parameters are available and take a look at the instrument geometry.

In [ ]:
instrument.show_parameters()

In [ ]:
# instrument.show_instrument(format='window')
instrument.show_diagram()

### Question 1
Question about what is going on in the instrument model, checking for example how many choppers there are

In [ ]:
def plot(data, var1="th", var2="t", t_bins=200, **kwargs):
    event1 = ms.name_search("Banana_large", data)
    event2 = ms.name_search("Banana_small", data)

    bins1=(200, t_bins)
    bins2=(100, t_bins)
    
    ms.make_sub_plot([event2.make_2d(var1, var2, n_bins=bins2), event1.make_2d(var1, var2, n_bins=bins1)], **kwargs)

def plot_opposite(data, var1="th", var2="t", t_bins=200, **kwargs):
    event1 = ms.name_search("Banana_large", data)
    event2 = ms.name_search("Banana_small", data)

    bins1=(200, t_bins)
    bins2=(100, t_bins)
    
    ms.make_sub_plot([event1.make_2d(var1, var2, n_bins=bins1), event2.make_2d(var1, var2, n_bins=bins2)], **kwargs)

In [ ]:
mpi = 8

### Run basic instrument
Run without pulse shaping chopper

In [ ]:
instrument.get_component("chopper").set_WHEN("frequency_multiplier!=0") # code to remove chopper

In [ ]:
# instrument.set_parameters(frequency_multiplier=3, guide_curve_deg=0, detector_height=1.5)
# instrument.settings(ncount=5e8, mpi=8, suppress_output=False, NeXus=True, output_path="powder_run")

instrument.settings(ncount=2e8, mpi=mpi, suppress_output=True, NeXus=True, output_path="powder_run")
instrument.set_parameters(sample_choice='"sample_fe"', frequency_multiplier=3, guide_curve_deg=0, detector_height=1.5)


data = instrument.backengine()
data

In [ ]:
plot(data, orders_of_mag=5, log=True)

### Run basic instrument
Here with pulse shaping chopper running too slow

In [ ]:
instrument.set_parameters(reflections='"Fe.laz"', frequency_multiplier=1, guide_curve_deg=0, detector_height=1.5)
instrument.settings(ncount=5e8, mpi=14, suppress_output=True, NeXus=True, output_path="powder_run")

data = instrument.backengine()
data

In [ ]:
plot(data, orders_of_mag=5, log=True)

## Question 2


In [ ]:

instrument.settings(ncount=2e8, mpi=mpi, suppress_output=True, NeXus=True, output_path="powder_run")

instrument.set_parameters(sample_choice='"sample_Si"', frequency_multiplier=1, guide_curve_deg=0, detector_height=1.5)


instrument.set_parameters(frequency_multiplier=0)
data_without = instrument.backengine()

instrument.set_parameters(frequency_multiplier=1)
data_with = instrument.backengine()

instrument.set_parameters(frequency_multiplier=3)
data_with_fast = instrument.backengine()

In [ ]:
plot_options = dict(orders_of_mag=5, log=True, left_lim=[-170, 10], right_lim=[-10, 170])

plot(data_without, **plot_options)
plot(data_with, **plot_options)
plot(data_with_fast, **plot_options)

In [ ]:
plot_options = dict(orders_of_mag=5, log=True, left_lim=[10, -170], right_lim=[170, -10], top_lim=0.16, bottom_lim=0.06)

plot_opposite(data_without, **plot_options)
plot_opposite(data_with, **plot_options)
plot_opposite(data_with_fast, **plot_options)

In [ ]:
data_without[0].original_data_location

In [ ]:

instrument.settings(ncount=2e8, mpi=mpi, suppress_output=True, NeXus=True, output_path="powder_run")

instrument.set_parameters(sample_choice='"sample_2"', frequency_multiplier=1, guide_curve_deg=0, detector_height=1.5)


instrument.set_parameters(frequency_multiplier=0)
data_without = instrument.backengine()

instrument.set_parameters(frequency_multiplier=1)
data_with = instrument.backengine()

instrument.set_parameters(frequency_multiplier=3)
data_with_fast = instrument.backengine()

In [ ]:
plot_options = dict(orders_of_mag=5, log=True, left_lim=[-170, 10], right_lim=[-10, 170])

plot(data_without, **plot_options)
plot(data_with, **plot_options)
plot(data_with_fast, **plot_options)

In [ ]:

instrument.settings(ncount=2e8, mpi=mpi, suppress_output=True, NeXus=True, output_path="powder_run")

instrument.set_parameters(sample_choice='"sample_vanadium"', frequency_multiplier=1, guide_curve_deg=0, detector_height=1.5)


# instrument.set_parameters(frequency_multiplier=0)
# data_without = instrument.backengine()

# instrument.set_parameters(frequency_multiplier=1)
# data_with = instrument.backengine()

instrument.set_parameters(frequency_multiplier=3)
data_with_fast = instrument.backengine()

In [ ]:
plot_options = dict(orders_of_mag=5, log=True, left_lim=[-170, 10], right_lim=[-10, 170])

# plot(data_without, **plot_options)
# plot(data_with, **plot_options)
plot(data_with_fast, **plot_options)

In [ ]:
data_with_fast[0].original_data_location

In [ ]:

t_without = ms.name_search("Banana_small", data_without).make_1d("th", n_bins=50)
t_with = ms.name_search("Banana_small", data_with).make_1d("th", n_bins=50)
t_with_fast = ms.name_search("Banana_small", data_with_fast).make_1d("th", n_bins=50)

ms.make_sub_plot([t_without, t_with, t_with_fast])

## Set parameters

Before we run a simulation using the instrument, we need to set some parameters to the desired values.

## Instrument settings
Before running the simulation, a few settings pertaining to computing options need to be specified. This is done with a different method to clearly distinguish these from the instrument parameters. One important setting is called `output_path` which sets the name of the generated folder with simulation output.

In [ ]:
instrument.set_parameters(reflections='"Fe.laz"', frequency_multiplier=3, guide_curve_deg=0, detector_height=1.5)
instrument.settings(ncount=5e8, mpi=4, suppress_output=True, NeXus=True, output_path="powder_run")

In [ ]:
instrument.settings(mpi=2)

## Run the instrument
Now the simulation can be executed with the `backengine` method. This method returns a list of data object. Store this returned data in a python variable named `data`.

In [ ]:
data = instrument.backengine()
data

### Visualize the data
The data objects in the returned list can be plotted with the McStasScript function `make_plot`. The plots can be customized, use these keyword arguments:
- log : True
- orders_of_mag : 5 (maximum orders of magnitudes plotted when using logarithmic plotting)

In [ ]:
ms.make_sub_plot(data, log=True, orders_of_mag=5)

In [ ]:
data

In [ ]:
event1 = ms.name_search("Banana_large", data)
event2 = ms.name_search("Banana_small", data)

In [ ]:
ms.make_sub_plot([event2.make_2d("th", "y"), event1.make_2d("th", "y")], log=True)

In [ ]:
event1 = ms.name_search("Banana_large", data)
event2 = ms.name_search("Banana_small", data)

var1 = "th"
var2 = "t"
t_bins = 200
bins1=(100, t_bins)
bins2=(50, t_bins)

ms.make_sub_plot([event2.make_2d(var1, var2, n_bins=bins2), event1.make_2d(var1, var2, n_bins=bins1)], log=True, orders_of_mag=5)

In [ ]:
event1.variables

In [ ]:
event1.get_data_column("U1")

In [ ]:
travel_time = event1.get_data_column("t") - event1.get_data_column("U1")

In [ ]:
travel_distance = event1.get_data_column("v")*travel_time

In [ ]:
travel_distance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(travel_distance, 100)

## Improve the instrument


## Run improved instrument

## Run with sample
